In [2]:
import pandas as pd

import math
from math import pi, pow, sin, cos, asin, sqrt, floor
from scipy import stats
import numpy as np


In [11]:
df = pd.read_csv('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/Assignment_3_Redo/Data/pop_data.csv')

df= df[['pop2020', 'name']]

df.to_csv('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/Assignment_3_Redo/Data/pop_data_2020.csv', index=False)

In [108]:
def parse_illinois_counties_to_adjacency_list(file_path):
    adjacency_list = {}
    current_county = None
    
    with open(file_path, 'r') as file:
        for line in file:
            # Remove leading/trailing whitespace and split by tab
            parts = line.strip().split("\t")
            
            if len(parts) >= 2 and "IL" in line and line.startswith('"'):
                # Main county is the first part
                county_name = parts[0].strip('"').replace(", IL", "")
                current_county = county_name
                
                # Initialize the adjacency list for the current county
                if county_name not in adjacency_list:
                    adjacency_list[county_name] = []
                
                # Loop through the rest of the parts (neighbor counties)
                
                neighbor_county = parts[2].strip('"').replace(", IL", "")
                if neighbor_county != current_county:
                    adjacency_list[county_name].append(neighbor_county)

            elif current_county and "IL" in line:
                # Only add neighboring counties in Illinois and remove ", IL"
                neighbor_county = line.strip().split("\t")[0].strip('"').replace(", IL", "")
                # Add the neighbor only if it's not the same as the current county
                if neighbor_county != current_county:
                    adjacency_list[current_county].append(neighbor_county)
    
    return adjacency_list

# Parse the adjacency data file
adj_list = parse_illinois_counties_to_adjacency_list('/Users/gracefujinaga/Documents/Northwestern/MSDS_460/redistricting/Assignment_3_Redo/Data/adjacency_data.txt')

# Optionally print out the adjacency list for debugging
for county, neighbors in adj_list.items():
    print(f"{county}: {neighbors}")


Adams County: ['Brown County', 'Hancock County', 'Pike County', 'Schuyler County']
Alexander County: ['Pulaski County', 'Union County']
Bond County: ['Clinton County', 'Fayette County', 'Madison County', 'Montgomery County']
Boone County: ['DeKalb County', 'McHenry County', 'Ogle County', 'Winnebago County']
Brown County: ['Adams County', 'Cass County', 'Morgan County', 'Pike County', 'Schuyler County']
Bureau County: ['Henry County', 'LaSalle County', 'Lee County', 'Marshall County', 'Putnam County', 'Stark County', 'Whiteside County']
Calhoun County: ['Greene County', 'Jersey County', 'Pike County']
Carroll County: ['Jo Daviess County', 'Ogle County', 'Stephenson County', 'Whiteside County']
Cass County: ['Brown County', 'Mason County', 'Menard County', 'Morgan County', 'Sangamon County', 'Schuyler County']
Champaign County: ['Douglas County', 'Edgar County', 'Ford County', 'McLean County', 'Piatt County', 'Vermilion County']
Christian County: ['Macon County', 'Montgomery County', 'S

In [57]:
print(len(adj_list))

102


In [37]:
for county in ['Cook County', 'Lake County', 'DuPage County']:
    adj_list.pop(county)

In [50]:

def check_adjacency(counties_to_test, adj_list):
    # Iterate over each pair of counties in the list
    for i, county1 in enumerate(counties_to_test):
        for county2 in counties_to_test[i+1:]:  # Check each pair only once
            # Check if county2 is in the adjacency list of county1 or vice versa
            if county2 in adj_list.get(county1, []):
                print(f"{county1} and {county2} are adjacent.")
            elif county1 in adj_list.get(county2, []):
                print(f"{county1} and {county2} are adjacent.")
            else:
                print(f"{county1} and {county2} are NOT adjacent.")

# List of counties to test adjacency
counties_to_test = ["Bureau County", "Marshall County", "Putnam County", "Will County"]


# Call the function with your adjacency list
check_adjacency(counties_to_test, adj_list)


Bureau County and Marshall County are adjacent.
Bureau County and Putnam County are adjacent.
Bureau County and Will County are NOT adjacent.
Marshall County and Putnam County are adjacent.
Marshall County and Will County are NOT adjacent.
Putnam County and Will County are NOT adjacent.


In [77]:
for county in counties_to_test: 
    print(county)
    print(adj_list.get(county))
    

Bureau County
['Henry County', 'LaSalle County', 'Lee County', 'Marshall County', 'Putnam County', 'Stark County', 'Whiteside County']
Marshall County
['LaSalle County', 'Peoria County', 'Putnam County', 'Stark County', 'Woodford County']
Putnam County
['LaSalle County', 'Marshall County']
Will County
['DuPage County', 'Grundy County', 'Kankakee County', 'Kendall County']


In [78]:
counties_str = "Bureau County, Marshall County, Putnam County, Will County"


counties = counties_str.split(", ")


check_adjacency(counties, adj_list)

Bureau County and Marshall County are adjacent.
Bureau County and Putnam County are adjacent.
Bureau County and Will County are NOT adjacent.
Marshall County and Putnam County are adjacent.
Marshall County and Will County are NOT adjacent.
Putnam County and Will County are NOT adjacent.


In [123]:
counties_str = "Bond County, Champaign County, Christian County, Clinton County, Coles County, Cumberland County, Ford County, Johnson County, LaSalle County, Lee County, Livingston County, Macon County, Montgomery County, Perry County, Putnam County, Union County, Washington County"


counties = counties_str.split(", ")

def filter_adjacency_list(adj_list, filter_keys):
    # Create a new dictionary to store filtered adjacency list
    filtered_adj_list = {}
    
    # Loop through the adjacency list
    for county, neighbors in adj_list.items():
        # Check if the county is in the filter_keys list
        if county in filter_keys:
            # Keep the county and its neighbors
            # Filter neighbors to only include those also in the filter_keys list
            filtered_neighbors = [neighbor for neighbor in neighbors if neighbor in filter_keys]
            filtered_adj_list[county] = filtered_neighbors
    
    return filtered_adj_list

# Example usage:
# Assuming adj_list is your original adjacency list and you have a list of counties to filter
filter_keys = counties  # The counties you want to keep
filtered_adj_list = filter_adjacency_list(adj_list, filter_keys)

# Print the filtered adjacency list
print(filtered_adj_list)



{'Bond County': ['Clinton County', 'Montgomery County'], 'Champaign County': ['Ford County'], 'Christian County': ['Macon County', 'Montgomery County'], 'Clinton County': ['Bond County', 'Washington County'], 'Coles County': ['Cumberland County'], 'Cumberland County': ['Coles County'], 'Ford County': ['Champaign County', 'Livingston County'], 'Johnson County': ['Union County'], 'LaSalle County': ['Lee County', 'Livingston County', 'Putnam County'], 'Lee County': ['LaSalle County'], 'Livingston County': ['Ford County', 'LaSalle County'], 'Macon County': ['Christian County'], 'Montgomery County': ['Bond County', 'Christian County'], 'Perry County': ['Washington County'], 'Putnam County': ['LaSalle County'], 'Union County': ['Johnson County'], 'Washington County': ['Clinton County', 'Perry County']}


In [124]:
# Function to perform DFS and find connected components
def find_connected_components(adjacency_list):
    visited = set()  # Keeps track of visited counties
    connected_components = []  # List to hold all connected components

    def dfs(county, component):
        """Depth First Search to explore connected counties"""
        visited.add(county)  # Mark the current county as visited
        component.append(county)  # Add the county to the current component
        
        # Explore all unvisited neighbors
        for neighbor in adjacency_list.get(county, []):
            if neighbor not in visited:
                dfs(neighbor, component)

    # Iterate over all counties in the adjacency list
    for county in adjacency_list:
        if county not in visited:  # If the county hasn't been visited
            component = []  # Start a new connected component
            dfs(county, component)  # Explore the connected component
            connected_components.append(component)  # Add the component to the list
    
    return connected_components


# Find connected components
connected_components = find_connected_components(filtered_adj_list)

# Print the connected components
for i, component in enumerate(connected_components, 1):
    print(f"Connected Component {i}: {', '.join(component)}")


Connected Component 1: Bond County, Clinton County, Washington County, Perry County, Montgomery County, Christian County, Macon County
Connected Component 2: Champaign County, Ford County, Livingston County, LaSalle County, Lee County, Putnam County
Connected Component 3: Coles County, Cumberland County
Connected Component 4: Johnson County, Union County


In [99]:
adj_list.get('Clinton County')

['Bond County',
 'Fayette County',
 'Madison County',
 'Marion County',
 'St. Clair County',
 'Washington County']